<a href="https://colab.research.google.com/github/kiyong21c/kaggle/blob/main/20220609_cat-in-dat-modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 피처 엔지니어링
 - 머신러닝 모델은 문자 데이터를 인식하지 못함
 - 문자 → 숫자 : 인코딩
 

## 데이터 합치기

In [154]:
import pandas as pd

data_path = '/content/drive/MyDrive/Colab Notebooks/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [155]:
all_data = pd.concat([train, test]) # 훈련데이터와 테스트 데이터 합치기
all_data = all_data.drop('target', axis=1) # 훈련데이터셋에 있던 타깃값 제거
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,Green,Square,Lion,Canada,Theremin,6c90f81cd,a406a5f12,d02a6b0ba,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3
499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,Piano,2d61990e2,520806ce2,d2d8eabdb,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2
499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,Oboe,488406659,28645754b,22831fffe,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1


 - **문자로 구성된 피처는 반드시 인코딩**
 - 하지않으면 모델 훈련 불가
 - 숫자로 구성된 피처도 성능향상을 위해 인코딩하기도 함
 - 여기서는 편의상 모든 피처 인코딩

In [156]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder() # 원-핫 인코더 생성
all_data_encoded = encoder.fit_transform(all_data) # 원-핫 인코딩 적용

## 데이터 나누기
 - 공통으로 인코딩 적용 후 다시 훈련데이터와 테스트 데이터 나누기
 - 훈련데이터에서 다시 일부를 검증데이터로 나누기

In [172]:
num_train = len(train) # 300,000

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']

 - 훈련데이터와 검증데이터를 나눌때는 train_test_split() 함수 사용
 - 타깃값 공정 배분 여부, 훈련/검증 데이터 비율, 데이터 무작위 추출 등을 설정 할 수 있기때문

In [173]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1, # 10%만 검증데이터로 배분
                                                      stratify=y, # y 즉, 타깃값이 훈련/검증 데이터 각각에 같은 비율로 포함되게끔 배분
                                                      random_state=10)

## 모델 훈련
 - 로지스틱 회귀 모델 : 분류 모델
 

In [175]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42)
history = logistic_model.fit(X_train, y_train)

## 모델 성능 검증
 - predict() : 타깃값 자체(0 또는 1)를 예측
 - predict_proba() : 타깃값의 확률(0일 확률, 1일 확률)을 예측

In [191]:
logistic_model.predict_proba(X_valid) # 첫번째 열은 0, 두번째 열은 1일 확률

array([[0.2327832 , 0.7672168 ],
       [0.91405072, 0.08594928],
       [0.83025789, 0.16974211],
       ...,
       [0.24862988, 0.75137012],
       [0.49420097, 0.50579903],
       [0.9565535 , 0.0434465 ]])

In [194]:
logistic_model.predict(X_valid) # 높은 확률의 타깃값을 예측

array([1, 0, 0, ..., 1, 1, 0])

 - 본 대회에서는 타깃값이 1일 '확률'을 예측해야함
 - 따라서 predict_proba() 메서드로 예측한 결과의 두번째 열을 타깃 예측값으로 사용

In [195]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

 - 타깃 예측값인 y_valid_preds와 실제 타깃값인 y_valid를 이용해 ROC AUC를 구함 : 모델 성능 검증
 - ROC AUC 점수는 사이킷런 : roc_auc_score() 함수 이용

In [196]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

roc_acu = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_acu:.4f}') # 소수점 4자리 까지 표시

검증 데이터 ROC AUC : 0.7965


## 예측 및 결과 제출
 - 검증 데이터가 아닌 실제 테스트 데이터 활용해 타깃값 1일 확률 예측

In [197]:
y_preds = logistic_model.predict_proba(X_test)[:, 1]

 - 훈련을 훈련세트의 90%만 썻음..
 - 검증세트로 10% 떼어 놓음 : 성능에 영향은 없는지?


In [198]:
submission['target'] = y_preds
submission.to_csv(data_path + 'submission.csv')

 - 프라이빗 점수, 퍼블릭 점수가 낮게 나옴
 - 다음 절에서 모델 성능을 향상 시켜봄

# 성능 개선
 - 피처 엔지니어링 : 피처 맞춤 인코딩, 피처 스케일링
 - 하이퍼 파라미터 최적화 : 그리드 서치